<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark

In [2]:
!pip install pyspark
!pip install findspark

import findspark
findspark.init()
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegressionModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 25.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=6866435556f988b8a678802eec64818bb486e71edb361e6c5da6420653e1c4da
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/06/51/98/f7a41aad64c08302d6c26c90650e713c3dfeb5cdec4946db00
Successfully built pyspark


In [3]:
# Start session

In [4]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("SparkML Ops").getOrCreate()

23/05/01 16:34:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [6]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"

--2023-05-01 16:34:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.3’

searchterms.csv.3   100%[===================>] 227.99K  --.-KB/s    in 0.005s  

2023-05-01 16:34:13 (42.4 MB/s) - ‘searchterms.csv.3’ saved [233457/233457]



In [7]:
# Load the csv into a spark dataframe

In [8]:
df = pd.read_csv("searchterms.csv")
spark_df = spark.createDataFrame(df)

In [9]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [10]:
rows = spark_df.count()
cols = len(spark_df.columns)
print(f"Number of rows: {rows}")
print(f"Number of columns: {cols}")

[Stage 0:=======>                                                   (1 + 7) / 8]

Number of rows: 10000
Number of columns: 4


In [11]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [12]:
spark_df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [13]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [14]:
spark_df.select("searchterm").printSchema()

root
 |-- searchterm: string (nullable = true)



In [15]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [16]:
spark_df.createOrReplaceTempView("searchterms")

spark.sql("select count(*) as gaming_laptop from searchterms where searchterm = 'gaming laptop'").show()

[Stage 3:>                                                          (0 + 8) / 8]

+-------------+
|gaming_laptop|
+-------------+
|          499|
+-------------+



In [17]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [18]:
spark.sql("select searchterm, count(*) as count from searchterms group by searchterm order by count(*) desc limit 5").show()

[Stage 6:==========================================>            (156 + 8) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+



In [19]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [20]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [21]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
!tar -xvzf model.tar.gz

model = LinearRegressionModel.load("sales_prediction.model")

--2023-05-01 16:34:41--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz.3’

model.tar.gz.3      100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-05-01 16:34:41 (9.37 MB/s) - ‘model.tar.gz.3’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/

In [22]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [23]:
def predict (year):
    assembler = VectorAssembler(inputCols = ["year"], outputCol = "features")
    data = [[year, 0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select("features", "year")
    predictions = model.transform(__)
    predictions.select("prediction").show()
    
predict(2023)

[Stage 13:======================================>                   (2 + 1) / 3]

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/05/01 16:34:52 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/01 16:34:52 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
